# Causal Machine Learning for Treatment Effect Estimation

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.cluster import KMeans

from sklearn.preprocessing import Normalizer

# get standard models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPRegressor

from sources.doubly_robust import doubly_robust
from sources.models.TARNet import TARnetICFR

from significance import *

import pickle
import torch
import torch.nn as nn
import torch.optim as optim

from auxiliary import data_processing, doubly_robust, visualise, visualise_ites, impute_missing_values_knn, run_model, undersample, full_contra_indications_tracker, value_based_contra_indications_tracker, period_decomposition, multicol
from icodes import encoding

# from datetime import timedelta
import warnings 
warnings.filterwarnings('ignore')

## 1. Introduction

The purpose of this notebook is to provide and end-to-end description of the treatment effect estimation process. This notebook consists of three parts:

1. **Data processing**: The first part is the data processing part. Here we basically prepare the data for the ML models.
2. **Causal ML Models**: The second part of the notebook consists of a series of causal ML learning models. In particular, we use the causal meta-learning framework. 
3. **Adadptive Model**: The third part of the notebook is also a machine learning model. However, contrary to the models before, this one was developed by the VU team and allows for more control over the estimation proces. 

Please find below an overview of the steps we will go through. Note that the boxes do not correspond one-to-one to sections in the notebook, but the general flow is the same. 

![alt text](images\ModellingOverview.jpg)

## 1.a. Preliminaries \& Design Choices

We first want to get some preliminaries out of the way. In particular, we would like to make some specific **design choices** explicit. Below, we can set the following parameters, thereby modifying specific choices:

- ``EXPOSURE_THRESHOLD``: The treatment is binary. However, how much treatment is provided can be changed with this variable. In essence, it determines the cut-off value. For instance, we can set it to 60 minutes in the case of physical therapy (in mins.). Then, patients with more then 60 minutes of PT will be assigned T=1, those below will be T=0.
- ``PERIOD_MIN``: Minimal time period between baseline measurement and followup measurement. N.B. If this is set too low, the effect may not yet register. Too high and the effect may have faded out. 
- ``PERIOD_MAX``: Maximal time period between baseline measurement and followup measurement. N.B. If this is set too low, effect may not yet register. Too high and the effect may have faded out.  
- ``REMOVE_MULTI_COL``: Whether or not to remove multicollinear columns. 
- ``CORRELATION_THRESHOLD``: Threshold for removing collinear columns. 
- ``UNDERSAMPLE``: Some models perform better when undersampling the minority class. For instance, if there are 1000 cases of patients with PT (T=0) and only 150 with PT (T=1), setting this to ``True`` ensures that 150 patients are sampled from PT (T=0), resulting in 300 observations in total.
- ``IMPUTE``: Whether or not to impute missing *covariate* values.
- ``PROPENSITY``: Whether or not to add a propensity score to the model.
- ``CLEAN_COMPARE``: By default (when this is set to ``False``), the data processing script assigns any observations below ``EXPOSURE_THRESHOLD`` to the control group and  any observation above ``EXPOSURE_THRESHOLD`` to treated. Setting this parameter to ``True`` creates a scenario where *only* observation that did not get any treatment are assigned to the control group. For instance, in the case of PT, we would have a control group of patients getting only 0 mins. of PT and a treated group of patients with, say, more than 60 mins. of PT.
- ``TREATMENT``: Name of the treatment variable.
- ``TARGET``: Name of the target variable.

In [2]:
# set seed
np.random.seed(42)

# set the exposure threshold for binary dichtomization. In the case of minor social activities, more than 30 days ago. 
EXPOSURE_THRESHOLD = 1#30 # SET TO 30

# set the period between observations. We only consider the effects between 120 and 240 days after exposure. 
PERIOD_MIN = 120 # /MAYBE CHECK WITH 60? 
PERIOD_MAX = 240 # CHECK WITH 120 DAYS 

# if we want to move multicollinear columns, set to True
REMOVE_MULTI_COL = False

# set the threshold for multicollinearity drops 
CORRELATION_THRESHOLD = 0.6

# set to True if we want to undersample
UNDERSAMPLE = False #False #True

# set to True if we want to impute missing values
IMPUTE = False

PROPENSITY = True

# clean compare (to be used if only > threshold AND treatment = 0)
CLEAN_COMPARE = True

TREATMENT = 'is2l' # (days of Social activities)
OUTCOME = 'sCPS' 

In [3]:
#df

Having set the desired parameters, we will now read in the data.

In [4]:
file = "data/dutch_LTCF_all.csv"

# read data
df = pd.read_csv(file)
print(f"Treatment before processing {len(df[df[TREATMENT]>=EXPOSURE_THRESHOLD])}")

Treatment before processing 15603


In [5]:
#example how to created new variable is2l.cat from is2l
# new variable is2l.cat will take values 1 (if is2l=1 or 3 or 4) or 0 (if is2l=0 or is2l=2)
df['is2l.cat']=df['is2l'].apply(lambda x: 0 if(x==0 or x==2) else 1 if (x==1 or x==3 or x==4) else None)
# Control
pd.crosstab(df['is2l'],df['is2l.cat'])

is2l.cat,0.0,1.0
is2l,,
0.0,27830,0
1.0,0,851
2.0,2366,0
3.0,0,5285
4.0,0,7101


In [6]:
#example how to created new variable ia4.cat from ia4
#ia4= marital status
# 1= never married, 2= married, 3= partner/significant other, 4= widowed, 5= seperated,6= divorced
# new variable ia4.cat will take values 1 (if ia4=2 or 3) or 0 (if ia4= 1 or 4 or 5 or 6)
df['ia4.cat']=df['ia4'].apply(lambda x: 1 if(x==2 or x==3) else 0 if (x==1 or x==4 or x==5 or x==6) else None)
# Control
pd.crosstab(df['ia4'],df['ia4.cat'])

ia4.cat,0.0,1.0
ia4,,
1.0,5703,0
2.0,0,9089
3.0,0,325
4.0,22256,0
5.0,57,0
6.0,2045,0


In [7]:
#example how to create a new variable for DRS ii1c.cat from ii1c
#ii1c= Alzheimers disease
# 0= not present, 1= Primary Diagnosis for current stay 2= Diagnosis present receiving active treatment,3=diagnosis presesent, monitored, not receiving active treatment
# new variable ii1c.cat will take values 1 (if ii1c=1 or 2 or 3 ) or 0 (if ii1c=0)
df['ii1c.cat']=df['ii1c'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ii1c'],df['ii1c.cat'])

ii1c.cat,0.0,1.0
ii1c,,
0.0,30683,0
1.0,0,3055
2.0,0,3389
3.0,0,6632


In [8]:
#example how to create a new variable for DRS ii1d.cat from ii1d
#ii1d= Dementia disease
# 0= 0= not present, 1= Primary Diagnosis for current stay, 2= Diagnosis present receiving active treatment,3=diagnosis presesent, monitored, not receiving active treatment
# new variable ii1d.cat will take values 1 (if ii1d=1 or 2 or 3 ) or 0 (if ii1d=0)
df['ii1d.cat']=df['ii1d'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ii1d'],df['ii1d.cat'])

ii1d.cat,0.0,1.0
ii1d,,
0.0,33713,0
1.0,0,2721
2.0,0,2365
3.0,0,4920


In [9]:
#ii1o= Depression
df['ii1o.cat']=df['ii1o'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ii1o'],df['ii1o.cat'])

ii1o.cat,0.0,1.0
ii1o,,
0.0,35752,0
1.0,0,1268
2.0,0,4503
3.0,0,2193


In [10]:
#ii1o= Coronary heart disease
df['ii1k.cat']=df['ii1k'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ii1k'],df['ii1k.cat'])

ii1k.cat,0.0,1.0
ii1k,,
0.0,40136,0
1.0,0,460
2.0,0,1940
3.0,0,1140


In [11]:
#ii1t= Diabetes
df['ii1t.cat']=df['ii1t'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ii1t'],df['ii1t.cat'])

ii1t.cat,0.0,1.0
ii1t,,
0.0,33886,0
1.0,0,556
2.0,0,8186
3.0,0,1127


In [12]:
#ib3b= Ethenicity
#df['ib3b.cat']=df['ib3b'].apply(lambda x: 0 if(x==4) else 1 if (x==1 or x==2 or x==3) else None)
# Control
#pd.crosstab(df['ib3b'],df['ib3b.cat'])

In [13]:
#iA12a= living arrangements
df['ia12a.cat']=df['ia12a'].apply(lambda x: 0 if(x==1) else 1 if (x==4 or x==2 or x==3 or x==5 or x==6 or x==7 or x==8 or x==9) else None)
# Control
pd.crosstab(df['ia12a'],df['ia12a.cat'])

ia12a.cat,0.0,1.0
ia12a,,
1.0,30315,0
2.0,0,12121
3.0,0,883
4.0,0,1150
5.0,0,121
6.0,0,343
7.0,0,271
8.0,0,1234


In [14]:
#ii1m= COPD
df['ii1m.cat']=df['ii1m'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ii1m'],df['ii1m.cat'])

ii1m.cat,0.0,1.0
ii1m,,
0.0,38205,0
1.0,0,597
2.0,0,3709
3.0,0,1195


In [15]:
#ii1s= Cancer
df['ii1s.cat']=df['ii1s'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ii1s'],df['ii1s.cat'])

ii1s.cat,0.0,1.0
ii1s,,
0.0,39487,0
1.0,0,781
2.0,0,925
3.0,0,2554


In [16]:
#ii1l= Congestive heart disease
df['ii1l.cat']=df['ii1l'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ii1l'],df['ii1l.cat'])

ii1l.cat,0.0,1.0
ii1l,,
0.0,32395,0
1.0,0,1175
2.0,0,7788
3.0,0,2323


In [17]:
#ii1j= Stroke 
df['ii1j.cat']=df['ii1j'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ii1j'],df['ii1j.cat'])

ii1j.cat,0.0,1.0
ii1j,,
0.0,34378,0
1.0,0,1520
2.0,0,4060
3.0,0,3787


In [18]:
#ij8a= smoke
df['ij8a.cat']=df['ij8a'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2) else None)
# Control
pd.crosstab(df['ij8a'],df['ij8a.cat'])

ij8a.cat,0.0,1.0
ij8a,,
0.0,39039,0
1.0,0,349
2.0,0,4381


In [19]:
#ij8b= Alcohol
df['ij8b.cat']=df['ij8b'].apply(lambda x: 0 if(x==0 or x==1) else 1 if (x==3 or x==2) else None)
# Control
pd.crosstab(df['ij8b'],df['ij8b.cat'])

ij8b.cat,0.0,1.0
ij8b,,
0.0,34334,0
1.0,5437,0
2.0,0,2893
3.0,0,1067


In [20]:
#iG8a= Change in ADL 
df['iG8a2.cat']=df['iG8a2'].apply(lambda x: 0 if(x==1 or x==2 or x==8) else 1 if (x==0) else None)
# Control
pd.crosstab(df['iG8a2'],df['iG8a2.cat'])

iG8a2.cat,0.0,1.0
iG8a2,,
0.0,0,7199
1.0,14346,0
2.0,5058,0
8.0,1679,0


In [21]:
#example how to created new variable if1a.cat from if1a
# 0=never, 1= more than 30 days, 2= 8-30 days, 3-4-7 days, 4= in  last 3 days
# new variable if1a.cat will take values 1 (if if1a=1 or 2 or 3 or 4) or 0 (if if1a=0)
df['if1c.cat']=df['if1c'].apply(lambda x: 0 if(x==0) else 1 if (x==1 or x==2 or x==3 or x==4) else None)
# Control
pd.crosstab(df['if1c'],df['if1c.cat'])

if1c.cat,0.0,1.0
if1c,,
0.0,13388,0
1.0,0,1637
2.0,0,2025
3.0,0,3031
4.0,0,18139


In [22]:
#iG8a= Change in ADL 
df['ie1k.cat']=df['ie1k'].apply(lambda x: 0 if(x==0 ) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ie1k'],df['ie1k.cat'])

ie1k.cat,0.0,1.0
ie1k,,
0.0,33033,0
1.0,0,5180
2.0,0,2561
3.0,0,2973


In [23]:
#ie1j= Reduced social interaction
df['ie1j.cat']=df['ie1j'].apply(lambda x: 0 if(x==0 ) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ie1j'],df['ie1j.cat'])

ie1j.cat,0.0,1.0
ie1j,,
0.0,28659,0
1.0,0,5255
2.0,0,3206
3.0,0,6698


In [24]:
#ie1i= withdrawl from activities of interest
df['ie1i.cat']=df['ie1i'].apply(lambda x: 0 if(x==0 ) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['ie1i'],df['ie1i.cat'])

ie1i.cat,0.0,1.0
ie1i,,
0.0,32054,0
1.0,0,4655
2.0,0,2642
3.0,0,4485


In [25]:
#if5a= at ease interacting with others
df['if5a.cat']=df['if5a'].apply(lambda x: 0 if(x==0 ) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['if5a'],df['if5a.cat'])

if5a.cat,0.0,1.0
if5a,,
0.0,3678,0
1.0,0,2921
2.0,0,5759
3.0,0,31417


In [26]:
#if5a= at ease interacting with others
df['if5a.cat']=df['if5a'].apply(lambda x: 0 if(x==0 ) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['if5a'],df['if5a.cat'])

if5a.cat,0.0,1.0
if5a,,
0.0,3678,0
1.0,0,2921
2.0,0,5759
3.0,0,31417


In [27]:
#if5b= at ease interacting with others
df['if5b.cat']=df['if5b'].apply(lambda x: 0 if(x==0 ) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['if5b'],df['if5b.cat'])

if5b.cat,0.0,1.0
if5b,,
0.0,7894,0
1.0,0,5363
2.0,0,5922
3.0,0,24571


In [28]:
#if5c= at ease interacting with others
df['if5c.cat']=df['if5c'].apply(lambda x: 0 if(x==0 ) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['if5c'],df['if5c.cat'])

if5c.cat,0.0,1.0
if5c,,
0.0,13294,0
1.0,0,7815
2.0,0,7669
3.0,0,14970


In [29]:
#if5d= at ease interacting with others
df['if5d.cat']=df['if5d'].apply(lambda x: 0 if(x==0 ) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['if5d'],df['if5d.cat'])

if5d.cat,0.0,1.0
if5d,,
0.0,17881,0
1.0,0,5317
2.0,0,5112
3.0,0,15436


In [30]:
#if5e= at ease interacting with others
df['if5e.cat']=df['if5e'].apply(lambda x: 0 if(x==0 ) else 1 if (x==1 or x==2 or x==3) else None)
# Control
pd.crosstab(df['if5e'],df['if5e.cat'])

if5e.cat,0.0,1.0
if5e,,
0.0,10130,0
1.0,0,4516
2.0,0,6036
3.0,0,23080


In [31]:
#df.to_excel("excel_version.xlsx")

In [32]:
# code for date

for i in df.columns:
    print(i)

Unnamed: 0
Clientid
Snapshotid
qrt
qrt2
Clientid_original
ClientDepartmentcode
facility_id_first
facility_id
Facility_country
origin
LOS
sum_total
valid_total
Instrument
iA0
iA2
ia4
Age
sAGE_cat
iA6b
ia8
iB2
iA9
yr_assessed
ia11a
ia11b
ia12a
ia13
ib3a
iB4
ib5a
ib5b
ib5c
ib5cnl
ib5d
ib5e
ib6
ib7
ic1
ic4
ic5
ic2a
ic2b
ic2c
ic2c1nl
ic2c2nl
ic2d
ic3a
ic3b
ic3c
id1
id2
id3a
id3b
id4a
id4b
ie1a
ie1b
ie1c
ie1d
ie1e
ie1f
ie1g
ie1h
ie1i
ie1j
ie1k
ie2a
ie2b
ie2c
ie3a
ie3b
ie3c
ie3d
ie3e
ie3f
if4
if1a
if1b
if1c
if1d
if5a
if5b
if5c
if5d
if5e
if5f
if5g
if6a
if6b
if7a
if7b
if7c
if7d
if8a
if8b
if8c
ig3
ig4
ig5
iG8a2
ig12
ig2a
ig2b
ig2c
ig2d
ig2e
ig2f
ig2g
ig2h
ig2i
ig2j
ig6a
ig6b
ig7a
ig7b
ig8a
ih1
ih2
ih3
ih4
ih5
ii1a
ii1b
ii1c
ii1cc
ii1d
ii1dd
ii1e
ii1f
ii1g
ii1h
ii1i
ii1j
ii1k
ii1l
ii1m
ii1n
ii1o
ii1p
ii1q
ii1r
ii1s
ii1t
ii1w
ij1
ij3
ij4
ij7
ij12
ij2a
ij2b
ij2c
ij2d
ij2e
ij2f
ij2g
ij2h
ij2i
ij2j
ij2k
ij2l
ij2m
ij2mm
ij2n
ij2o
ij2p
ij2q
ij2r
ij2s
ij2t
ij5a
ij5b
ij5c
ij5d
ij5e
ij6a
ij6b
ij6c
ij8a
ij

In [33]:
code = 'in5a'
print(encoding[(code).lower()])

Inpatient acute hospital with overnight stay: # times past 90 days


In [34]:
#df['iA9']
# This iA9  means date 

In [35]:
code = 'in5a'
print(encoding[(code).lower()])

Inpatient acute hospital with overnight stay: # times past 90 days


In [36]:
code = 'ib5a'
print(encoding[(code).lower()])

5 Year History-Long Term Care Facility


In [37]:
#code = 'if5a'
#print(encoding[(code).lower()])

In [38]:
code = 'iB4'
print(encoding[(code).lower()])

Primary language


In [39]:
total_samples=len(df)

In [40]:
print(f'Total number of samples:{total_samples}')

Total number of samples:48142


In [41]:
len(df['Clientid'].unique())

12785

## Part 2. Data Processing

In this first part, we will carry out the data processsing. In particular, we would like to select the relevant covariates, make a pre-selection based on patient characteristics (e.g. whether or not contra-indications are present), select the relevant period of observation, impute missing *coviariate* values and, lastly, handle any multi-collinearity. 

Next, we define the lists of covariates, clinical indications and contra-indications

In [42]:
          
# list relevant confounders
confounders = ['ia4.cat','if1c.cat', 'ii1c.cat', 'ii1d.cat','ia12a', 'ib5e','ib5c','ig6a', 'id3a','id4a','ij8a.cat', 'ij8b.cat','ii1m.cat',
               'ii1s.cat', 'ii1l.cat', 'ii1j.cat','ig2b', 'ig2f', 'ig2h','ig2j','ie1a','ie1b', 'ie1c', 'ie1d', 'ie1e', 'ie1f', 'ie1g',
                        'iG8a2','ii1o.cat', 'ii1k.cat', 'ii1t.cat','if1d','iA2', 'sAGE_cat', 'in3fc', 'ie1k.cat','if5a.cat','if5b.cat','if5c.cat',
              'if5d.cat','if5e.cat']

#also convert values to list for ease of processing later on 
#listed_val_based_con = [key for key, value in value_based_confounders.items()]
#confounders = confounders =+ Listed_val_based_confounders
        
# ADD VALUE BASED CLINICAL INDICATIONS, E.G. {'sDRS' : [3,4]}

# list of other relevant variables
relevant_vars = ['iA9', 'Clientid', TREATMENT, OUTCOME]

print(len(confounders))

41


In [43]:
for col in confounders:
    df[col]

Please note that you can get the meaning of the icodes by running the ``encoding``-function below. You only need to change the name of the ``code`` variable.

In [44]:
code = 'in5a'
print(encoding[(code).lower()])

Inpatient acute hospital with overnight stay: # times past 90 days


In [45]:
df['facility_id_first']

0        42420220.0
1        75750563.0
2        42420220.0
3        42420220.0
4        42420220.0
            ...    
48137           NaN
48138           NaN
48139           NaN
48140           NaN
48141           NaN
Name: facility_id_first, Length: 48142, dtype: float64

Finally, for some contra-indications we want to exclude based on particular values only. We can do this by running the script below. 

In [46]:
# df = df[list(confounders + relevant_vars)]
# print(f"Treatment after selecting covariates {len(df[df[TREATMENT]>=EXPOSURE_THRESHOLD])}")

In [47]:
df = df[list(confounders + relevant_vars)]
print(f"Treatment after selecting covariates {len(df[df[TREATMENT]>=EXPOSURE_THRESHOLD])}")

Treatment after selecting covariates 15603


In [48]:
df[list(confounders + relevant_vars)]

,ia4.cat,if1c.cat,ii1c.cat,ii1d.cat,ia12a,ib5e,ib5c,ig6a,id3a,id4a,...,ie1k.cat,if5a.cat,if5b.cat,if5c.cat,if5d.cat,if5e.cat,iA9,Clientid,is2l,sCPS
0,0.0,1.0,0.0,0.0,1.0,NaN,NaN,4.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,2005-06-13,42,4.0,1.0
1,1.0,NaN,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,2006-06-09,43,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,2006-01-13,82,4.0,2.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,2005-06-06,82,4.0,2.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,2005-06-13,83,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48137,NaN,1.0,0.0,0.0,1.0,NaN,0.0,3.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,2022-11-10,1825793914,4.0,0.0
48138,0.0,0.0,0.0,1.0,1.0,NaN,0.0,3.0,1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,1.0,2022-11-10,3998986670,0.0,5.0
48139,NaN,0.0,0.0,0.0,1.0,NaN,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,2022-11-10,6135579548,1.0,4.0
48140,NaN,1.0,0.0,1.0,1.0,NaN,0.0,4.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,2022-11-10,8479471735,4.0,1.0


In [49]:
df.head()

,ia4.cat,if1c.cat,ii1c.cat,ii1d.cat,ia12a,ib5e,ib5c,ig6a,id3a,id4a,...,ie1k.cat,if5a.cat,if5b.cat,if5c.cat,if5d.cat,if5e.cat,iA9,Clientid,is2l,sCPS
0,0.0,1.0,0.0,0.0,1.0,NaN,NaN,4.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,2005-06-13,42,4.0,1.0
1,1.0,NaN,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,2006-06-09,43,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,2006-01-13,82,4.0,2.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,2005-06-06,82,4.0,2.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,2005-06-13,83,0.0,NaN


### 2.b. Baseline-Followup Selection

Important to note here is that in this particular setup, our goal is to **estimate the outcome value at follow up from the covariates *and* outcome variable at baseline**. 

Below you find a schematic depiction of how we want to process the data. As you can see, there are two timelines:

1. **Timeline 1**: Naturally processes from baseline to follow up over time, without any intervention in the meantme.
2. **Timeline 2**: Same as before, but with the difference that not an intervention has take place at some point in time between baseline and follow up. 

![alt text](TimelineModels.jpg)

Our goal is therefore to, *for each observation*, take the values (**pre-treatment** covariates, outcome on baseline $y_{t}$) of baseline, and the  values at follow up (outcome on follow up $y_{t+1}$) and store it in a convenient manner. Important to note is that as patient may have *several* of these baseline-followup recors where sometimes an intervention may have occured and sometimes not. We decided to do this to, again, safe as much data as possible. 

Let us begin by first counting the number of assesments. We do this, because it is not possible to include patients without followup. 

In [50]:
# get number of assesments
counter = lambda x: len(df[df['Clientid']==x])

# count number of items
df['num_assesments'] = df['Clientid'].apply(counter)

Let us select only observations with more than 2 assesments. 

In [51]:
# get number of assesments higher than 1
df = df[df['num_assesments']>=2]

print(f'{len(df)} observations remaining.')
print(f"Treatment after selecting on number of assesments {len(df[df[TREATMENT]>=EXPOSURE_THRESHOLD])}")

44613 observations remaining.
Treatment after selecting on number of assesments 14856


In the code below, we want to process the data a bit further. First, we want to make sure that the date columns (``iA9``) is in the correct format. Then we want to order based on ID and date. Also, we want to drop missing dates (if there are any) and drop any duplicated observations. Lastly, we want to make sure that the outcome is in the right format (i.e. a ``float``).

In [52]:
# convert column to datetime 
df['iA9'] = pd.to_datetime(df['iA9']) 
# sort values by ID and date
df = df.sort_values(by = ['Clientid', 'iA9'])
# drop nans on dates of assesment
df = df[df['iA9'].isna()==False]
# drop duplicated values
df = df.drop_duplicates()
df[OUTCOME] = df[OUTCOME].astype(float)

### 2.c Drop Duplicated Values

Next, because there may be two assesments on the same date, we also want to drop those (regardless of the reason as inspecting that is beyond the scope of this project notebook). 

In [53]:
# Group by 'Clientid' and then apply the duplicate check on 'iA9' within each group
duplicated_indices = df.groupby('Clientid').apply(
    lambda x: x[x['iA9'].duplicated()].index).explode()

# Drop NaN values from the index list if they exist
duplicated_indices = duplicated_indices.dropna()

# Convert the result into a list if it's not empty
if not duplicated_indices.empty:
    duplicated_indices = duplicated_indices.tolist()

    # Drop the duplicates using the list of indices
    df = df.drop(index=duplicated_indices)

### 2.d. Clean Compare
If we want to only compare between those patients that receive either 0 treatment or a treatment above the threshold value, we run this code.

In [54]:
if CLEAN_COMPARE:
    df = df[(df[TREATMENT] <= 0) | (df[TREATMENT] >= EXPOSURE_THRESHOLD)]

### 2.e Period Decomposition

The piece of code (i.e. the function ``period_decomposition``) below is a loop that basically does the following:

1. Create a temporary dataframe ``temp_df``. We do this to make sure that there is an empty entity (i.e. data storage) to which we can append new, cleaned and processed observations.
2. Then, for each patient we do the following:
3. Look at all the observation dates.
4. Select the target outcome at follow up and at baseline.
5. Append each period to the ``temp_df`` *until running out of dates*. 
6. Move to the next patient. 

In [55]:
def period_decomposition(df, target):
    # List to store the rows of the new DataFrame
    rows = []

    # Unique client IDs
    client_ids = df['Clientid'].unique()
    total = len(client_ids)

    for count, client_id in enumerate(client_ids, start=1):
        # Inform the user about the progress
        if count % 1000 == 0 or count == total:
            print(f'{count} of {total} items completed...')

        # Extract rows for the current client
        client_rows = df[df['Clientid'] == client_id].sort_values('iA9')
        client_dates = client_rows['iA9'].tolist()

        for i in range(len(client_dates) - 1):
            baseline_date = client_dates[i]
            followup_date = client_dates[i + 1]

            # Extract the rows for baseline and follow-up
            baseline_row = client_rows[client_rows['iA9'] == baseline_date]
            followup_row = client_rows[client_rows['iA9'] == followup_date]
            

            # Calculate the outcomes
            outcome_t0 = float(baseline_row[target])
            outcome_t1 = float(followup_row[target])

            # Prepare a new row with all necessary information
            new_row = baseline_row.iloc[0].to_dict()
            new_row['OutcomeT0'] = outcome_t0
            new_row['OutcomeT1'] = outcome_t1
            new_row['OutcomeT0Date'] = baseline_date
            new_row['OutcomeT1Date'] = followup_date
            rows.append(new_row)

    # Create a new DataFrame from the list of new rows
    temp_df = pd.DataFrame(rows)
    print("Completed.")
    return temp_df


In [56]:
df = period_decomposition(df, target = OUTCOME)
print(f"Treatment after processing {len(df[df[TREATMENT]>EXPOSURE_THRESHOLD])}")

1000 of 9120 items completed...
2000 of 9120 items completed...
3000 of 9120 items completed...
4000 of 9120 items completed...
5000 of 9120 items completed...
6000 of 9120 items completed...
7000 of 9120 items completed...
8000 of 9120 items completed...
9000 of 9120 items completed...
9120 of 9120 items completed...
Completed.
Treatment after processing 10895


### 2.f Dichotomize Treatment Variable

Because our model can only deal with binary treatments, we need to convert the continuous treatments into binary ones. We will do that using a simple fuctions. Recall that the threshold has been set using the ``EXPOSURE_THRESHOLD`` variable above.

In [57]:
#df[TREATMENT]

In [58]:
binary = lambda x: 1 if x >= EXPOSURE_THRESHOLD else 0

In [59]:
#df[TREATMENT].apply(binary)

In [60]:
df[TREATMENT]=df[TREATMENT].apply(binary)

In [61]:
df['treatment']=df[TREATMENT]

In [62]:
df=df.drop(columns=[TREATMENT])

In [63]:
# binary = lambda x: 1 if x >= EXPOSURE_THRESHOLD else 0
# convert treatment to binary
# df['treatment'] = df[TREATMENT].apply(binary)  
# df = df.drop(columns = [TREATMENT])

In [64]:
#df

In [65]:
# drop rows with missing outcome or treatment 
df = df.dropna(subset = ['OutcomeT0', 'OutcomeT1', 'treatment'])
df.head(3)

,ia4.cat,if1c.cat,ii1c.cat,ii1d.cat,ia12a,ib5e,ib5c,ig6a,id3a,id4a,...,if5e.cat,iA9,Clientid,sCPS,num_assesments,OutcomeT0,OutcomeT1,OutcomeT0Date,OutcomeT1Date,treatment
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2005-06-06,82,2.0,2,2.0,2.0,2005-06-06,2006-01-13,1
3,1.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,...,1.0,2005-07-01,195,2.0,2,2.0,2.0,2005-07-01,2006-02-14,1
4,0.0,1.0,0.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,...,1.0,2005-06-06,219,1.0,2,1.0,0.0,2005-06-06,2006-01-13,0


In [66]:
#df

In [67]:
year_to_filter = 2014
treatment_count = len(df[(df['iA9'].dt.year == year_to_filter) & (df['treatment'] ==1)])

total_count = len(df[df['iA9'].dt.year == year_to_filter ])

treatment_percentage = (treatment_count / total_count)*100
print(treatment_percentage)

26.96774193548387


In [68]:
year_to_filter = 2015
treatment_count = len(df[(df['iA9'].dt.year == year_to_filter) & (df['treatment'] ==1)])

total_count = len(df[df['iA9'].dt.year == year_to_filter ])

treatment_percentage = (treatment_count / total_count)*100
print(treatment_percentage)

30.697674418604652


In [69]:
year_to_filter = 2016
treatment_count = len(df[(df['iA9'].dt.year == year_to_filter) & (df['treatment'] ==1)])

total_count = len(df[df['iA9'].dt.year == year_to_filter ])

treatment_percentage = (treatment_count / total_count)*100
print(treatment_percentage)

39.625167336010705


In [70]:
year_to_filter = 2017
treatment_count = len(df[(df['iA9'].dt.year == year_to_filter) & (df['treatment'] ==1)])

total_count = len(df[df['iA9'].dt.year == year_to_filter ])

treatment_percentage = (treatment_count / total_count)*100
print(treatment_percentage)

39.421965317919074


In [71]:
year_to_filter = 2018
treatment_count = len(df[(df['iA9'].dt.year == year_to_filter) & (df['treatment'] ==1)])

total_count = len(df[df['iA9'].dt.year == year_to_filter ])

treatment_percentage = (treatment_count / total_count)*100
print(treatment_percentage)

44.68325791855204


In [72]:
year_to_filter = 2019
treatment_count = len(df[(df['iA9'].dt.year == year_to_filter) & (df['treatment'] ==1)])

total_count = len(df[df['iA9'].dt.year == year_to_filter ])

treatment_percentage = (treatment_count / total_count)*100
print(treatment_percentage)

38.866930171278


In [73]:
year_to_filter = 2020
treatment_count = len(df[(df['iA9'].dt.year == year_to_filter) & (df['treatment'] ==1)])

total_count = len(df[df['iA9'].dt.year == year_to_filter ])

treatment_percentage = (treatment_count / total_count)*100
print(treatment_percentage)

35.49234135667396


In [74]:
year_to_filter = 2021
treatment_count = len(df[(df['iA9'].dt.year == year_to_filter) & (df['treatment'] ==1)])

total_count = len(df[df['iA9'].dt.year == year_to_filter ])

treatment_percentage = (treatment_count / total_count)*100
print(treatment_percentage)

34.49709060681629


In [75]:
year_to_filter = 2022
treatment_count = len(df[(df['iA9'].dt.year == year_to_filter) & (df['treatment'] ==1)])

total_count = len(df[df['iA9'].dt.year == year_to_filter ])

treatment_percentage = (treatment_count / total_count)*100
print(treatment_percentage)

32.21216041397154


In [76]:
year_to_filter = 2023
treatment_count = len(df[(df['iA9'].dt.year == year_to_filter) & (df['treatment'] ==1)])

total_count = len(df[df['iA9'].dt.year == year_to_filter ])

treatment_percentage = (treatment_count / total_count)*100
print(treatment_percentage)

44.44444444444444


### 2.g Select Relevant Period

Because the base data is too crude (the time periods between follow-up and baseline may be too far apart), we want to select a suitable period between observations. In essence, we aim to select a time window which is, on the one hand, determined by the minimum time (``PERIOD_MIN``) between baseline measurement and follow-up measurement and the maximum of that time window (``PERIOD_MAX``).

In [77]:
# get difference between dates
df['date_diff'] = df['OutcomeT1Date'] - df['OutcomeT0Date'] 
df['date_diff'] = df['date_diff'].dt.days 

# select relevant period
df = df[df['date_diff'] <= PERIOD_MAX]
df = df[df['date_diff'] >= PERIOD_MIN]

#df = df.drop(columns = ['OutcomeT1Date', 'OutcomeT0Date', 'date_diff', 'iA9', OUTCOME, 'Clientid'])
# examine how many treatment observations are left
len(df[df['treatment']==1])

5785

In [78]:
df.head()

,ia4.cat,if1c.cat,ii1c.cat,ii1d.cat,ia12a,ib5e,ib5c,ig6a,id3a,id4a,...,iA9,Clientid,sCPS,num_assesments,OutcomeT0,OutcomeT1,OutcomeT0Date,OutcomeT1Date,treatment,date_diff
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2005-06-06,82,2.0,2,2.0,2.0,2005-06-06,2006-01-13,1,221
3,1.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,...,2005-07-01,195,2.0,2,2.0,2.0,2005-07-01,2006-02-14,1,228
4,0.0,1.0,0.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,...,2005-06-06,219,1.0,2,1.0,0.0,2005-06-06,2006-01-13,0,221
6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2005-06-13,226,1.0,2,1.0,0.0,2005-06-13,2006-01-17,1,218
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,...,2005-06-13,233,1.0,2,1.0,0.0,2005-06-13,2006-01-15,1,216


In [79]:
grps = df[['treatment', 'Clientid']].drop_duplicates()

In [80]:
print(f'Number of patients after processing (total): {len(grps)}')

Number of patients after processing (total): 5766


In [81]:
tr = len(grps[grps['treatment']==1])
ct = len(grps[grps['treatment']==0])

In [82]:
print(f'Number of patients after processing (treatment): {ct}')
print(f'Number of patients after processing (treatment): {tr}')

Number of patients after processing (treatment): 3641
Number of patients after processing (treatment): 2125


In [83]:
df = df.drop(columns = ['OutcomeT1Date', 'OutcomeT0Date', 'date_diff', 'iA9', OUTCOME, 'Clientid'])

### 2.h Impute Missing Values

In this part, we impute missing values. We use a standard KNN-imputer. If desired, it is possible to change the number of neighbours. Keep in mind that we can set this to ``False`` if we do not want to run it,

In [84]:
#imputing = IMPUTE
if IMPUTE:
    df = impute_missing_values_knn(df, n_neighbors=5)
else:
    df = df.dropna()
    print(len(df[df['treatment']==1]))

2517


### 2.i Handle Multicollinearity

Here we will handle multi-collinear columns. Keep in mind that we can set this to ``False`` if we do not want to run it,

In [85]:
if REMOVE_MULTI_COL:
    print('2sr')
    df = multicol(df, CORRELATION_THRESHOLD)

### 2.j Handle Empty Columns

In the process, it may be that we end up with columns that contain only $0$ values. We want to drop these columns, as they are not informative.

In [86]:
# drop num_assesment column, not relevant anymore
df = df.drop(columns = ['num_assesments'])
# get a list of coviariate columns
X = df.drop(columns = ['OutcomeT1', 'OutcomeT0', 'treatment']).columns

# drop columns that only have 0 values
for col in X:
    if len(df[df[col]==0]) == len(df):
        print(f"{col} - {encoding[col.lower()]} dropped." )
        df = df.drop(columns = [col])

### 2.k Store CSV

In the last part of this notebook, we will store that data.

In [87]:
X = df.drop(columns = ['OutcomeT1', 'OutcomeT0', 'treatment']).columns

In [88]:
# treatment = TREATMENT#'in3eb' # (minutes or days of physical therapy)
# target = #'sADLSF' 

#df = df.drop(columns = 'num_assesments')
df.to_csv(f"data/03-10-2023-Dutch_LTCF_cleaned_data_with_selected_covar_{TREATMENT}-{OUTCOME}.csv", index = False)

In [89]:
# ADD SKIP FOR TREATMENT AND OUTCOME T1
covars = [str(i) for i in df.columns]
covar_names = []

#print(encoding[(code).lower()])

for covar in covars:
    if (covar != 'treatment') and (covar != 'OutcomeT1'):

        try:
            covar_names.append(encoding[(covar).lower()])
        except:
            covar_names.append(covar)
            
            

In [90]:
covar_names

['catagories of Marital Status, 1 = 2 or 3= (Married), else= 0',
 'if1c.cat',
 'catagories of Alzheimers disease, 0= not present, else= 1',
 'catagories of Dementia other than Alzheimers disease, 0= not present, else= 1',
 'Living arrangement',
 '5 Year History-Mental health residence psychiatric group home',
 'ib5c',
 'Total hours of exercise or physical activity in last 3 days',
 'Hearing: ability to hear (with hearing appliance normally used)',
 'Ability to see in adequate light (with glasses or with other visual appliance normally used)',
 'ij8a.cat',
 'ij8b.cat',
 'ii1m.cat',
 'ii1s.cat',
 'ii1l.cat',
 'ii1j.cat',
 'ADL: Personal hygiene',
 'ADL: Locomotion',
 'ADL: Toilet use',
 'ADL: Eating',
 'Made negative statements',
 'Persistent anger with self or others',
 'Expressions, including nonverbal, of what appear to be unrealistic fears',
 'Repetitive health complaints',
 'Repetitive anxious complaints or concerns (non-health related)',
 'Sad, pained, or worried facial expressions

In [91]:
# ADD SKIP FOR TREATMENT AND OUTCOME T1
covars = [str(i) for i in df.columns]
covar_names = []

#print(encoding[(code).lower()])

for covar in covars:
    if (covar != 'treatment') and (covar != 'OutcomeT1'):

        try:
            covar_names.append(covar)
        except:
            covar_names.append(covar)

In [92]:
covar_names

['ia4.cat',
 'if1c.cat',
 'ii1c.cat',
 'ii1d.cat',
 'ia12a',
 'ib5e',
 'ib5c',
 'ig6a',
 'id3a',
 'id4a',
 'ij8a.cat',
 'ij8b.cat',
 'ii1m.cat',
 'ii1s.cat',
 'ii1l.cat',
 'ii1j.cat',
 'ig2b',
 'ig2f',
 'ig2h',
 'ig2j',
 'ie1a',
 'ie1b',
 'ie1c',
 'ie1d',
 'ie1e',
 'ie1f',
 'ie1g',
 'iG8a2',
 'ii1o.cat',
 'ii1k.cat',
 'ii1t.cat',
 'if1d',
 'iA2',
 'sAGE_cat',
 'in3fc',
 'ie1k.cat',
 'if5a.cat',
 'if5b.cat',
 'if5c.cat',
 'if5d.cat',
 'if5e.cat',
 'OutcomeT0']

In [93]:
print(f"Treatment group size before processing (observations): {len(df[df['treatment']==1])}")

Treatment group size before processing (observations): 2517


In [94]:
print(f"Control group size before processing (observations): {len(df[df['treatment']==0])}")

Control group size before processing (observations): 3586


In [97]:
# quick model
clf = Ridge()

# get data
X = df.drop(columns = ['OutcomeT1'])
y = df['OutcomeT1']

# train model
clf.fit(X, y)
importances = clf.feature_importances_
feature_names = [] #X.columns

# get col names
for col in X.columns:
    try:
        feature_names.append(encoding[(col).lower()])
    except:
        feature_names.append(col)

# get feature importance -> als je error krijgt, je hebt zelf icodes toegevoegd, dan weer aanpassen!

feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importances = feature_importances.sort_values(by = 'importance', ascending = False)

feature_importances

AttributeError: 'Ridge' object has no attribute 'feature_importances_'

### Note to Teams: Please Start Here

**MAKE SURE THAT THE DATA IS IN THE EXACT FORMAT AS BELOW.**

Checklist:

1. The covariates must come first.
2. Then, there must be the outcome at baseline called ``OutcomeT0``.
3. There must be an outcome at followup called ``OutcomeT1``.
4. You may have a propensity score columns (``ps``), but this is not required. Same for clusters.
5. There must be a treatment column *at the end* called ``treatment``. 

Good luck!

## Part 3. Machine Learning

Having processed the data, we can now move on the the machine learning (ML) part. Recall that we model using the meta-learning paradigm. Below is a schematic depiction of the process.

![alt text](images\ModelGoal.jpg)

As can be seen, for a given patient, we can - *after training* - provide the details of that patient. These details are the outcome at baseline (``OutcomeT0``), the covariates and whether or not we include treatment. This allows us to compute the **individual treatment effect** (ITE), the predicted effect unter treatment minus the predicted effect under control.

However, before we are able to estimate the ITE for a patient, we need to train the model. In meta-learning we always train two models. The first model is trained on the control group and the second model is trained on the treated group. Let us now first state what the ``outcome`` and ``intervention`` variables are.  

![alt text](images\GeneralPlotTrajectoryInterpretation.jpg)

Next, we want to select the models. Here we will use ``RandomForestRegressor()``.

In [ ]:
model0 = RandomForestRegressor()
model1 = RandomForestRegressor()

In [ ]:
model = model0

In [ ]:
# File path for the Excel file
excel_file = f'data/hyperparameters/model_hyperparameters_RandomForest.xlsx'

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
    # Get hyperparameters
    hyperparameters = model.get_params()

    # Convert to DataFrame
    df = pd.DataFrame([hyperparameters])

    # Write each model's hyperparameters to a different sheet
    df.to_excel(writer, sheet_name="RandomForest")

print("Hyperparameters of models have been saved to 'model_hyperparameters.xlsx'.")

Then, we want to retrieve the data. We can use the data from above, but here we will read the file. 

In [ ]:
# call your data file here
FILE = f"data/03-10-2023-Dutch_LTCF_cleaned_data_with_selected_covar_{TREATMENT}-{OUTCOME}.csv"

In [ ]:
#read the file
all_data = pd.read_csv(FILE)
all_data_control = all_data[all_data['treatment']==0]
all_data_treatment = all_data[all_data['treatment']==1]

In [ ]:
len(all_data)

In [ ]:
len(all_data_control)

In [ ]:
len(all_data_treatment) 

In [ ]:
percent_control = len(all_data_control)/len(all_data)
percent_control

In [ ]:
percent_treatment = len(all_data_treatment)/len(all_data)
percent_treatment

In [ ]:
len(all_data_treatment) 

In [ ]:
len(all_data_control)

In [ ]:
len(all_data)

In [ ]:
age_groups = {"60-64": (60,64)}
total_count = len(all_data)

age_filtered = all_data[(all_data['sAGE_cat'] >= 60) & (all_data['sAGE_cat'] <= 64)]

count_filtered = len(age_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)

print(count_filtered)

In [ ]:
age_groups = {"65-74": (65,74)}
total_count = len(all_data)

age_filtered = all_data[(all_data['sAGE_cat'] >= 65) & (all_data['sAGE_cat'] <= 74)]

count_filtered = len(age_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
age_groups = {"75-84": (75,84)}
total_count = len(all_data)

age_filtered = all_data[(all_data['sAGE_cat'] >= 75) & (all_data['sAGE_cat'] <= 84)]

count_filtered = len(age_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
age_groups = {"85-94": (85,94)}
total_count = len(all_data)

age_filtered = all_data[(all_data['sAGE_cat'] >= 85) & (all_data['sAGE_cat'] <= 94)]

count_filtered = len(age_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
total_count = len(all_data)

age_filtered = all_data[(all_data['sAGE_cat'] >= 95)]

count_filtered = len(age_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
#Calculate gender in total population
total_count = len(all_data)

Gender_filtered = all_data[(all_data['iA2'] == 1)]

count_filtered = len(Gender_filtered ) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
#Calculate gender in total population
total_count = len(all_data)

Gender_filtered = all_data[(all_data['iA2'] == 2)]

count_filtered = len(Gender_filtered ) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
#Calculat CPS(0-5) in total population,  excluding CPS = 6

total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT1'] == 0)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:

total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT0'] == 1)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT1'] == 1)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT0'] == 2)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT1'] == 2)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT0'] == 3)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT1'] == 3)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT0'] == 4)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)
print(count_filtered)

In [ ]:
total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT1'] == 4)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)

In [ ]:
total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT0'] == 5)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)

In [ ]:
total_count = len(all_data)

CPS_filtered = all_data[(all_data['OutcomeT1'] == 5)]

count_filtered = len(CPS_filtered) 

percentage = (count_filtered/total_count)*100

print(percentage)

Afterwards, we can run the model. The only thing to do is to run the ``run_model``-function below. 

In [ ]:
machine = "RandomForest"

metrics, predictions_t, predictions_c, y_test_t, y_test_c, ites_test, ites_train, X_test_t, X_test_c = run_model(n_bootstraps = 30, file = FILE, model0 = model0, model1 = model1, undersampled = UNDERSAMPLE, include_propensity = PROPENSITY, intervention = TREATMENT, outcome = OUTCOME, machine = "RandomForest", num_iter = 1)

We can now examine the performance of the model by calling ``metrics``.

In [ ]:
metrics

In [ ]:
confidence_interval(data=ites_test)

In [ ]:
significance(data=ites_test)

We can also visualise the model's *factual* predicted performance by calling the ``visualise`` function.

In [ ]:
len(y_test_t)

In [ ]:
machine = "RandomForest"
visualise(predictions_t, predictions_c, y_test_t, y_test_c, machine=machine, target=OUTCOME, intervention=TREATMENT)

Also, if we want to visualize the ITES, we can run the following function (``visualise_ites``):

In [ ]:
visualise_ites(ites_test, machine = machine, intervention = TREATMENT, target = OUTCOME)

In [ ]:
visualise_ites(ites_train, machine = machine, intervention = TREATMENT, target = OUTCOME)

### 3.b Run More Models

These models can be run also. Feel free to remove some of the models, but please run at least:

- ``RandomForestRegressor()``
- ``GradientBoostingRegressor()``
- ``LinearRegression()``

In [ ]:
model_lst = [(RandomForestRegressor(),RandomForestRegressor()), (MLPRegressor(), MLPRegressor()), 
            (GradientBoostingRegressor(), GradientBoostingRegressor()), (DecisionTreeRegressor(), DecisionTreeRegressor()),
            (LinearRegression(), LinearRegression()), (Ridge(), Ridge()), (Lasso(), Lasso()),
            (AdaBoostRegressor(), AdaBoostRegressor()), (ExtraTreesRegressor(), ExtraTreesRegressor()), 
             (BaggingRegressor(), BaggingRegressor())]

for model1, model2 in model_lst:
    print(str(model1))
    metrics, predictions_t, predictions_c, y_test_t, y_test_c, ites_test, ites_train, X_test_t, X_test_c = run_model(n_bootstraps = 30, file = FILE, include_propensity = PROPENSITY, outcome=OUTCOME, intervention=TREATMENT, undersampled = UNDERSAMPLE, model0 = model0, model1 = model1, machine = str(model1), num_iter = 1)
    #visualise(predictions_t, predictions_c, y_test_t, y_test_c, machine=machine, target=target, intervention=intervention)
    visualise_ites(ites_test, machine = str(model1), intervention = TREATMENT, target = OUTCOME)
    print(metrics)
    confidence_interval(data=ites_test)
    significance(data=ites_test)

## Part 4. Adaptive Model

In this last part, we will predict the treatment effect using an adaptive model. This model was developed by us. As can be seen, there is a bit more coding involved, but the benefit is that it is easier to adapt this model to our preferences. 

The first step is to make sure that our model runs on the proper device. 

In [ ]:
device = torch.device("cpu")

It is useful to have a small function that easily converts arrays to the correct ``torch``-objects: tensors.

In [ ]:
to_tensor = lambda x: torch.tensor(np.array(x), dtype = torch.float32).to(device)

In [ ]:
FILE = f"data/03-10-2023-Dutch_LTCF_cleaned_data_with_selected_covar_{TREATMENT}-{OUTCOME}.csv"
df = pd.read_csv(FILE)

### Uplift Model

In [ ]:
# Split into features and target
X = df.drop('OutcomeT1', axis=1)
y = df['OutcomeT1']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model on treatment group
model_treatment = LogisticRegression()
model_treatment.fit(X_train[X_train['treatment'] == 1].drop('treatment', axis=1), y_train[X_train['treatment'] == 1])

# Train model on control group
model_control = LogisticRegression()
model_control.fit(X_train[X_train['treatment'] == 0].drop('treatment', axis=1), y_train[X_train['treatment'] == 0])

# Predict probabilities
prob_treatment = model_treatment.predict_proba(X_test.drop('treatment', axis=1))[:, 1]
prob_control = model_control.predict_proba(X_test.drop('treatment', axis=1))[:, 1]

# Calculate uplift
uplift = prob_treatment - prob_control

# Sorting the individuals by predicted uplift
order = np.argsort(uplift)[::-1]
sorted_uplift = uplift[order]

# Calculating cumulative uplift
cumulative_uplift = np.cumsum(sorted_uplift) / len(y_test)

# Plotting
plt.figure(figsize=(10, 6))
plt.xlim(0,100)
plt.plot(cumulative_uplift, label='Uplift Curve')
plt.xlabel('Proportion Targeted')
plt.ylabel('Cumulative Uplift')
plt.title('Uplift Curve')
plt.legend()
plt.show()

In [ ]:
# apply min-max normalisation
#X = df.drop(columns = ['treatment', 'OutcomeT1']).columns
#df[X] = (df[X] - df[X].min()) / (df[X].max() - df[X].min())

In [ ]:
X = df.drop(columns = ['treatment', 'OutcomeT1']).columns

normalizer = Normalizer(norm="l2")
normalizer.fit(df[X])

df[X] = normalizer.fit_transform(df[X])

with open(f'storage/normalizer_{TREATMENT}-{OUTCOME}.pkl', 'wb') as f:
    pickle.dump(normalizer, f)

### 4 a. Perform Undersampling Procedure

In [ ]:
if UNDERSAMPLE:
    freq_treated = len(df[df['treatment']==1])
    control = df[df['treatment']==0].sample(freq_treated)
    treated = df[df['treatment']==1]
    df = pd.concat((control, treated))
    
print(f'DF lenght after undersampling: {len(df)}')

Because the model is a neural-network based architecture, it is usually helpful to apply min-max normalisation which converts the column values to \[0,1\] ranges.

### 4.b Train-Test Splitting

In the cell below, we perform train-test split. Note that we also split the train and test data into two groups: a control group and a treated group. Even though we do not explicItely use for train, it will be useful for the (sub-)group evaluation later on.

In [ ]:
# variable setting for treatment
t = 'treatment'
# select the relevant outcome
y = "OutcomeT1" 
# specify test size
test_size = 0.2

# Split groups into train and test
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = [y]), df[['treatment', y]], test_size=test_size, random_state=42)

### 4.c Propensity Score Computation

In [ ]:
# predict the propensity score
if PROPENSITY:
    X_train["OutcomeT1"] = y_train["OutcomeT1"]
    X_test["OutcomeT1"] = y_test["OutcomeT1"]
    X = X_train.drop(columns = ['treatment']).columns
    X = X_test.drop(columns = ['treatment']).columns
    ps_model = LogisticRegression(C=1e2, max_iter=10000).fit(X_train[X], y_train['treatment'])
    X_train['ps'] = ps_model.predict_proba(X_train[X])[:, 1]
    X_test['ps'] = ps_model.predict_proba(X_test[X])[:, 1]
    
    # ensure that treatment is last column
    X_train['treatment_new'] =  X_train['treatment']
    X_train = X_train.drop(columns = ['treatment'])
    X_train = X_train.rename(columns = {'treatment_new': 'treatment'})
    
    X_test['treatment_new'] =  X_test['treatment']
    X_test = X_test.drop(columns = ['treatment'])
    X_test = X_test.rename(columns = {'treatment_new': 'treatment'})
    
    #X_train = X_train.drop(columns = ["OutcomeT1"])
else:
    X_train["OutcomeT1"] = y_train["OutcomeT1"]

In [ ]:
X = df.drop(columns = ['OutcomeT1', 'treatment']).columns

#X_train = X_train.drop(columns = ["ps"])

# Estimate the average treatment effect (group level)
tau_est = doubly_robust(X_train, X=X, T='treatment', Y='OutcomeT1')
tau_est = torch.tensor(tau_est, dtype = torch.float32).to(device)
print(tau_est)

# drop columns
X_train = X_train.drop(columns = ["OutcomeT1"])

if PROPENSITY == True:
    X_test = X_test.drop(columns = ["OutcomeT1"])

In [ ]:
X_train

We also want to make sure that the data stuctures are in the right format. For this, we use ``to_tensor``.

In [ ]:
y_t = y_train[y_train['treatment']==1][y]
y_c = y_train[y_train['treatment']==0][y]

y_test_t = y_test[y_test['treatment']==1][y]
y_test_c = y_test[y_test['treatment']==0][y]

In [ ]:
# convert output to tensors
x, y, y_t, y_c, x_test = to_tensor(X_train), to_tensor(y_train), to_tensor(y_t), to_tensor(y_c), to_tensor(X_test)

### 4.d Model Training

The next step is to train the actual model. The code below sets some preliminary configurations.

In [ ]:
# Configurations
lr = 0.001
epochs = 1000
gamma = 2

model = TARnetICFR(x.shape[1], 0.01, hidden_dim = 32*4).to(device)

# Define the loss functions
head_loss = nn.L1Loss() #nn.MSELoss()#nn.L1Loss() #torch.nn.HuberLoss(reduction = 'mean', delta = 1) #nn.MSELoss() # nn.L1Loss() #nn.MSELoss() 

# initialise optimiser
optimizer = optim.Adam(model.parameters(), lr)

u = len(y_t)  / (len(y_t)  + len(y_c))

The code below is the training loop. On each loop, the model passes the data through the models, examines its performance and adjust the weights. Note that this model has four loss components:

1. Component 1 (``loss1``) is to loss on the treated segment of the data.
2. Component 2 (``loss2``) is to loss on the control segment of the data.
3. Component 2 (``loss_cf_3``) is to loss on the counterfactual control segment of the data.
4. Component 2 (``loss_cf_4``) is to loss on the counterfactual treated segment of the data.

In [ ]:
# Define loss lists
loss1_lst, loss2_lst, loss3_lst = [], [], []

for epoch in range(epochs):
    # Forward pass
    output_t, output_c, _ = model(x)
    
    # Compute total loss and update the model's parameters
    loss1, loss2 = head_loss(torch.squeeze(output_t), y_t), head_loss(torch.squeeze(output_c), y_c)
    
    pred_c_cf = model.forward_treated(x[x[:,-1]==0])
    loss_cf_3 = (1 - u) * torch.mean((torch.squeeze(pred_c_cf) - (y_c + tau_est))**2)
                 
    pred_t_cf = model.forward_control(x[x[:,-1]==1])
    loss_cf_4 = u * torch.mean((torch.squeeze(pred_t_cf) - (y_t - tau_est))**2)
    
    # losses added
    loss = loss1 + (loss2) + loss_cf_3 + loss_cf_4 #+ (gamma * loss3)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 10000 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch+1}/{epochs} | Loss: {loss.item():.4f}')
        
        # print intermediate results on test
        x_test_t = x_test[x_test[:,-1]==1] 
        x_test_c = x_test[x_test[:,-1]==0] 

        # # examine intermediate factual test performance
        y_t_pred = model.forward_treated(x_test_t).cpu().detach().numpy().reshape(len(x_test_t),)
        y_c_pred = model.forward_control(x_test_c).cpu().detach().numpy().reshape(len(x_test_c),)

        print(f"RMSE for factual treated {mean_squared_error(y_t_pred, y_test_t)}") 
        print(f"MAE for factual treated {mean_absolute_error(y_t_pred, y_test_t)}") 
        print(f"RMSE for factual control {mean_squared_error(y_c_pred, y_test_c)}") 
        print(f"MAE for factual treated {mean_absolute_error(y_c_pred, y_test_c)}") 

### 4.e Model Evaluation
Having trained the model, we can no examine the performance on the test set. We first get the predictions for the treated and control groups. Then we visualise the predicted predicted vs. true values. 

In [ ]:
y_t_pred = model.forward_treated(x_test[x_test[:,-1]==1] ).cpu().detach().numpy()
y_c_pred = model.forward_control(x_test[x_test[:,-1]==0] ).cpu().detach().numpy()

machine = "TARNet"
visualise(np.squeeze(y_t_pred), np.squeeze(y_c_pred), y_test_t, y_test_c, machine=machine, target=OUTCOME, intervention=TREATMENT)

Next, we want to estimate the model's predictions of the individual treatment effect. We plot this in an ordered fashion (from most beneficial to least beneficial). The blue dots depict patients. On the y-axis, we find the effect size as predicted by the model. 

In [ ]:
# isolate and disconnect predicted values
y_c_predt1 = model.forward_treated(x_test_c).cpu().detach().numpy()
y_t_predt1 = model.forward_treated(x_test_t).cpu().detach().numpy()
y_t_pred = np.concatenate((y_c_predt1, y_t_predt1))

# isolate and disconnect predicted values
y_c_predt0 = model.forward_control(x_test_c).cpu().detach().numpy() #np.array(y_test_c).reshape(-1,1) #model.forward_control(x_test_c).cpu().detach().numpy()
y_t_predt0 = model.forward_control(x_test_t).cpu().detach().numpy()

y_c_pred = np.concatenate((y_c_predt0, y_t_predt0))

# compute the ites and sort
ites = y_t_pred - y_c_pred
ites = [i[0] for i in ites]
ites = sorted(list(ites))

ate_model_est = np.mean(y_t_pred - y_c_pred)

print(f"ATE_est = {np.mean(y_t_pred)} - {np.mean(y_c_pred)} = {ate_model_est}.")

visualise_ites(np.array(ites), machine = machine, intervention = TREATMENT, target = OUTCOME)

In [ ]:
# isolate and disconnect predicted values
y_c_predt1 = model.forward_treated(x_test_c).cpu().detach().numpy()

# isolate and disconnect predicted values
y_c_predt0 = model.forward_control(x_test_c).cpu().detach().numpy() 

# compute the ites and sort
ites = y_c_predt1 - y_c_predt0
ites = [i[0] for i in ites]
ites = sorted(list(ites))

ate_model_est = np.mean(y_c_predt1 - y_c_predt0)

print(f"ATE_est = {np.mean(y_c_predt1)} - {np.mean(y_c_predt0)} = {ate_model_est}.")

visualise_ites(np.array(ites), machine = machine, intervention = TREATMENT, target = OUTCOME)

### 4.e Variable Importances

In [ ]:
np.random.seed(42)

# Calculate the baseline performance
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Ensure no gradients are computed
    output_t, output_c, _ = model(x)
baseline_performance = mean_squared_error(output_t, y_t)
print(baseline_performance)

# Initialize a list to hold the feature importances
feature_importances = []
icodes = []
meanings = []
rank = []

collist = X_train.columns

# Calculate the importance for each feature
for i in range(x.shape[1]):  # Iterate over each feature
    # Save the original feature
    original_feature = x[:, i].clone()
    # Permute the feature
    permuted_feature = original_feature[torch.randperm(original_feature.size(0))]
    x[:, i] = permuted_feature
    
    # Calculate performance with the permuted data
    with torch.no_grad():
        output_t, output_c, _ = model(x)
        
    permuted_performance_t = mean_squared_error(output_t, y_t)
    permuted_performance_c = mean_squared_error(output_c, y_c)
    
    permuted_performance = permuted_performance_t + permuted_performance_t
    
    # Calculate the importance as the change in performance
    importance = baseline_performance - permuted_performance
    feature_importances.append(importance)
    
    # Restore the original feature
    x[:, i] = original_feature
    
# Rank the features by their importance
sorted_features = np.argsort(feature_importances)[::-1]  # Indices of features, sorted by importance

# Print the feature importances
print("Feature importances:")
for i, feature_index in enumerate(sorted_features):
    #print(f"Feature {feature_index}: Importance {feature_importances[feature_index]}")
    icode = collist[feature_index]
    icodes.append(icode.lower())
    
    try:
        item = encoding[icode.lower()]
        #print(item)
        meanings.append(item)
    except KeyError:
        #print(f'{icode} not found')
        meanings.append(f'{icode} not found')
    #print("==================================================")
    
rankings = pd.DataFrame(icodes, columns = ['icodes'])
rankings['meanings'] = meanings
rankings['importance_values'] = feature_importances

rankings = rankings.sort_values(by = 'importance_values')
rankings = rankings.reset_index().drop(columns = 'index')
rankings

## Save Model

In [ ]:
model

In [ ]:
model = model
location = 'storage'

In [ ]:
# save model
with open(f'storage/model_{TREATMENT}-{OUTCOME}.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# load model
with open(f'storage/model_{TREATMENT}-{OUTCOME}.pkl', 'rb') as b:
    loaded_model = pickle.load(b)

In [ ]:
loaded_model